In [1]:
import mne
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,recall_score, f1_score
from processing_functions import ppt_id, create_epochs, freq_ind, create_numeric_labels, relative_band_power
warnings.simplefilter('ignore')

In [2]:
from sklearn.svm import SVC

We follow the same arrangement as in kNearestNeighbor folder to get the dataset.

In [3]:
ppt_diagnostics = pd.read_csv('data/ds004504/participants.tsv',sep='\t')
target_labels = ppt_diagnostics['Group'].apply(create_numeric_labels).values
target_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [4]:
epoch_length = 2000
overlap_ratio = 0.5
freq_bands = np.array([0.5,4.0,8.0,13.0,25.0,45.0])
sample_freq = 500 #hertz

In [5]:
features = []
targets = []
for i in range(len(target_labels)):
    ppt = i + 1
    raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
                                  + '/eeg/' + ppt_id(ppt) + '_task-eyesclosed_eeg.set', preload = True)
    export = raw_data.to_data_frame()
    ppt_array = export.iloc[:,range(1,len(export.columns))].values
    del raw_data
    del export
    ppt_epochs = create_epochs(ppt_array,epoch_length,overlap_ratio)
    freqs, ppt_psd  = welch(ppt_epochs,fs=sample_freq, axis=1)
    ppt_rbp = relative_band_power(ppt_psd,freqs,freq_bands)
    ppt_rbp_reshaped = ppt_rbp.reshape((ppt_rbp.shape[0], -1))
    features += [ppt_rbp_reshaped]
    targets += [[target_labels[i]]*ppt_rbp.shape[0]]

We used the same shuffle method as in kNN file.

In [30]:
CN_indices = list(i for i in range(37,66))
FTD_indices = list(i for i in range(66,89))
#np.random.shuffle(CN_indices)
#np.random.shuffle(FTD_indices)

In [40]:
print(CN_indices)
print(len(CN_indices))
print(FTD_indices)
print(len(FTD_indices))

[37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]
29
[66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88]
23


In [41]:
features = [features[CN_indices[i]-1] for i in range(0,29)] + [features[FTD_indices[i]-1] for i in range(0,23)]
targets = [targets[CN_indices[i]-1] for i in range(0,29)] + [targets[FTD_indices[i]-1] for i in range(0,23)]

In [48]:
#np.concatenate((features[0])).shape

In the next part, we will leave 1 patient out for each of the 52 (29 CN+ 23 FTD) patients and apply general SVM with kernel degree 2 polynomial for each case and calculate their accuracy, sensitivity, specificity and f1 scores.

In [50]:
cross_valid_acc = []
sensitivities = []
specificities = []
f1s = []
for i in range(52):
    # The following two arrays will store the target and features for the train set. We will append to it
    # as we go through various subjects.
    features_minus_i = ((features[0:52].copy()))
    del (features_minus_i)[i]
    
    targets_minus_i = ((targets[0:52].copy()))
    del (targets_minus_i)[i]
    
    train_X = np.concatenate(features_minus_i)
    train_y = np.concatenate(targets_minus_i)
    
    
    # The following two arrays will store target and features for the test set. 
    test_X = features[i]
    #np.concatenate((features[i],features[36+i]))
    test_y = targets[i]
    #np.concatenate((targets[i],targets[36+i]))
    
    #fitting the model
    
    svc = SVC(kernel='poly', degree=2, C=1000)
    svc.fit(train_X, train_y)
        
    # This stores the accuracy on the test data
    cross_valid_acc += [svc.score(test_X, test_y)]
    sensitivities += [recall_score(test_y, svc.predict(test_X), labels = [0,2], pos_label = 0)]
    specificities += [recall_score(test_y, svc.predict(test_X), labels = [0,2],pos_label = 2)]
    f1s += [f1_score(test_y, svc.predict(test_X),labels = [0,2],pos_label=0)]

In [51]:
np.mean(cross_valid_acc),np.std(cross_valid_acc)

(0.6518312173437895, 0.2897948197504423)

In [52]:
np.mean(sensitivities), np.std(sensitivities)

(0.4108151357133823, 0.40873149725919705)

In [53]:
np.mean(specificities), np.std(specificities)

(0.24101608163040716, 0.33903644239322284)

In [54]:
np.mean(f1s), np.std(f1s)

(0.4575848438867468, 0.43435068496319196)